# Manual ingest workflow, bypassing GUI

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, behavior, model
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import pathlib
from natsort import natsorted, ns
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-11-15 09:22:38,557][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-11-15 09:22:38,607][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.1'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.93.0'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
# from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment, model

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

some helpers

In [5]:

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_subject_key_from_dir(string):
    result = [item.split("_")[1] for item in string]
    return result

def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

## Parse directories

In [6]:
# get content of user directory
import fnmatch
dataroot = dj.config['custom']['exp_root_data_dir'][0]
# dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and '_' in d]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and fnmatch.fnmatch(d, 'JJ*') and fnmatch.fnmatch(d, '*11-*')]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
sorted_dirs_root

['JJ_ROS-1627_2023-11-03_scan9FKWTLNS_sess9FKWTLNS',
 'JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S',
 'JJ_ROS-1627_2023-11-03_scan9FKWT2RT_sess9FKWT2RT',
 'JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3',
 'JJ_ROS_1438_2022-11-22_scan9FF6TL96_sess9FF6TL96']

In [14]:
# sessi = "sess9FKW82R3"
# scansi = "scan9FKW82R3"
# sessi = "sess9FKWTLNS"
# scansi = "scan9FKWTLNS"

# sessi = "sess9FB2LN5C"
# scansi = "scan9FB2LN5C"

# scan9FJ842C3
# scan9FB2LN5C
# scan_key

scansi = "scan9FKWT95S"

scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
curation_key = (imaging.Curation & scan_key & 'curation_id=1').fetch1('KEY')
sessi = (scan.Scan & f'scan_id = "{scansi}"').fetch('session_id')[0]
session_key = (session.Session & f'session_id = "{sessi}"').fetch('KEY')[0]
aux_setup_typestr = (scan.ScanInfo() & scan_key).fetch("userfunction_info")[0] # check setup type (not needed)
print(aux_setup_typestr)
print((scan.ScanPath & scan_key).fetch("path")[0])

mini2p1_openfield
/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S


## Session ingest

In [15]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")
DLCModels = model.Model.fetch("model_name")

print("Project")
print(project)

print("Equipment")
print(Equipment)

print("Recording_Location")
print(Recording_Location)

print("s2pparm")
print(s2pparm)

print("DLCModels")
print(DLCModels)

Project
*project       project_descri
+------------+ +------------+
ATN            ATN-functional
dummy          dummy         
hpc-repstab    hpc-representa
rsc-functop    rsc-functional
rsc-hpc        rsc-hippocampa
rsc-latent     rsc-contextual
sc-lgn-actvis  sc-lgn-active-
V1-oddball     v1-oddball-pre
vc-lgn-repstab vc-lgn-represe
 (Total: 9)

Equipment
['bench2p' 'dummy' 'macroscope' 'mini2p_01' 'mini2p_02' 'mini2p_03'
 'mini2p_04' 'mini2p_05']
Recording_Location
['ATN' 'Ctx' 'dCA1' 'DG' 'dummy' 'LGNV1' 'RSCa' 'RSCg' 'V1']
s2pparm
[array([0, 1, 2, 3, 4, 5, 6]), array(['TR: Mini2p (new, non-rigid, individual scans, reg_tiff for movie, custom classifier)',
       'TR: Mini2p (rigid, mini2p classifier, no concat, REGTIFF)',
       'TR: Trondheim Mini2p (non-rigid, built-in classifier, scan_concat)',
       'TR: Bench2p (non-rigid, custom classifier, reg_tiff for movie, scans individually)',
       'TR: Bench2p (rigid, custom bench2p classifier, individual, SAVETIF)',
       'TR: Min

### Ingest Session and Scan

In [ ]:
isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
session.SessionSameSite.update1({'session_id': sessi, 'same_site_id': sessi})


In [11]:
query = scan.ScanPath() & 'scan_id = "' + scansi + '"'
dir_proc = query.fetch('path')[0]
print(dir_proc)

/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWTLNS_sess9FKWTLNS


In [ ]:
# POPULATE!
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 1}
scan.ScanInfo.populate(**populate_settings) 

In [ ]:

scan.Scan * scan.ScanInfo * session.SessionSameSite * session.Session() & f'session_id = "{sessi}"'

In [ ]:
# push scan to ProcessingTask
# TODO: handle multiscan concatenation from here?
selected_s2pparms_index = 0
imaging.ProcessingTask.insert1((sessi, scansi, selected_s2pparms_index, dir_proc, 'trigger'), skip_duplicates=True)

In [ ]:
imaging.ProcessingTask()

## Ingest AUX

In [ ]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]    
print(aux_setup_typestr)

In [ ]:
ibe.ingest_aux(sessi,scansi,verbose=True, aux_setup_type=aux_setup_typestr)

In [ ]:
(event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%frames%"')

In [ ]:
# time_differences = np.diff((event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%reward%"').fetch("event_start_time"))

# plt.figure(figsize=(10, 6))
# plt.hist(time_differences, bins=range(min(time_differences.astype(int)), max(time_differences.astype(int)) + 10, 1), edgecolor='black')
# plt.title('Histogram of Time Differences Between Consecutive Events')
# plt.xlabel('Time Difference (seconds)')
# plt.ylabel('Frequency')
# plt.show()

## Ingest BPOD


In [ ]:
ibe.ingest_bpod(sessi,scansi,verbose=False, aux_setup_type=aux_setup_typestr)

### get BPOD object to play around

In [ ]:
from adamacs.paths import get_imaging_root_data_dir, get_experiment_root_data_dir
from adamacs.ingest import bpod
from element_interface.utils import find_full_path

scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
bpod_path_relative = (scan.ScanPath & scan_key).fetch("path")[0]

bpod_path_full = list(find_full_path(
        get_experiment_root_data_dir(), bpod_path_relative
        ).glob("*mat"))[0]

In [ ]:
bpod_object = bpod.Bpodfile(bpod_path_full)

In [ ]:
bpod_object.session_data["Info"]

In [ ]:
bpod_object.session_data["MousePos"]['PreStimDLC_live'][10]['right_ear']

In [ ]:
bpod_object.trial_data[0]['Events']


In [ ]:

trials = bpod_object.trial_data
BNC1Low_events = [(i, trial['Events'].get('BNC1Low')) for i, trial in enumerate(trials) if 'BNC1Low' in trial['Events']]
print(BNC1Low_events)
# BNC1Low_events[0][0]

In [ ]:

trials[0]['Events']

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][1]["Events"]

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][0]["Events"]["BNC1High"]

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][-2]["Events"]

In [ ]:
BNC1Low_events = [(i, event.get('BNC1Low')) for i, event in enumerate(events) if 'BNC1Low' in event]

In [ ]:
BNC1Low_events

In [ ]:
bpod_object.session_data["StimPos"].keys()

In [ ]:
bpod_object.trial_data[34]['Events']

In [ ]:
trials = bpod_object.trial_data
SoftCode15_events = [(i, trial['Events'].get('SoftCode15')) for i, trial in enumerate(trials) if 'SoftCode15' in trial['Events']] # get all trials that have a BNC1Low event. Returns a list of tuples (trial number, event time)

SoftCode15_events_times = [x[1] for x in SoftCode15_events]

plt.hist(SoftCode15_events_times, bins=1000, edgecolor='black')
plt.xlim([0, 1])
plt.title('Histogram of softcode 15 event-to-trial-beginning time')
plt.xlabel('[s]')
plt.ylabel('Frequency')
# plt.xlim([0, 1])
plt.show()

In [ ]:
trials = bpod_object.trial_data
SoftCode10_events = [(i, trial['Events'].get('SoftCode10')) for i, trial in enumerate(trials) if 'SoftCode10' in trial['Events']] # get all trials that have a BNC1Low event. Returns a list of tuples (trial number, event time)

SoftCode10_events_times = [x[1] for x in SoftCode10_events]

plt.hist(SoftCode10_events_times, edgecolor='black')
plt.xlim([0, 1])
plt.title('Histogram of softcode 10 event-to-trial-beginning time')
plt.xlabel('[s]')
plt.ylabel('Frequency')
# plt.xlim([0, 1])
plt.show()

In [ ]:
SoftCode10_events_times

In [ ]:
1/60

In [ ]:
# get the times of the tone onsets
aux_tone = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
# create a time series of ones, using the same length as the tone onsets
onesfive = np.ones_like(aux_tone) -.5
# plot the time series of ones vs the tone onsets
axs[0].scatter(aux_tone, onesfive)

# get the times of the trial start events
bpod_trial = bpod_object.session_data["TrialStartTimestamp"]
# create a time series of ones, using the same length as the trial start events
ones = np.ones_like(bpod_trial)
# plot the time series of ones vs the trial start events
axs[1].scatter(bpod_trial, ones)

# get the times of the visual cue onsets
aux_vis = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
# create a time series of ones, using the same length as the visual cue onsets
onesvis = np.ones_like(aux_vis) -1
axs[2].scatter(aux_vis, onesvis)

# get the times of the auditory cue onsets
aux_aud = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%aud%"').fetch("event_start_time")
# create a time series of ones, using the same length as the auditory cue onsets
onesaud = np.ones_like(aux_aud) -2
axs[2].scatter(aux_aud, onesaud)

# get the times of the reward onsets

In [ ]:
def align_timeseries_and_find_non_matching_events(time_series_1, time_series_2):
    # Create event markers for each timeseries
    events_1 = np.zeros(int(np.ceil(max(time_series_1))))
    events_2 = np.zeros(int(np.ceil(max(time_series_2))))

    # Mark the events
    for event in time_series_1:
        events_1[int(event)] = 1
    for event in time_series_2:
        events_2[int(event)] = 1

    # Pad the shorter timeseries with zeros to make them the same length
    if len(events_1) > len(events_2):
        events_2 = np.pad(events_2, (0, len(events_1) - len(events_2)), 'constant')
    elif len(events_2) > len(events_1):
        events_1 = np.pad(events_1, (0, len(events_2) - len(events_1)), 'constant')

    # Perform cross-correlation
    correlation = correlate(events_1, events_2, mode='full')
    lag = np.argmax(correlation) - (len(events_2) - 1)

    # Shift the second series according to the lag
    aligned_series_2 = np.roll(events_2, -lag)[:len(events_1)]

    # Find the indices where events do not match
    non_matching_indices = np.where((events_1 == 1) & (aligned_series_2 == 0))[0]

    # Convert indices back to times
    non_matching_times = time_series_1[np.isin(time_series_1, non_matching_indices)]

    return non_matching_indices, non_matching_times, lag

# Align the two timeseries and find non-matching events
non_matching_indices, non_matching_times, alignment_lag = align_timeseries_and_find_non_matching_events(bpod_trial, aux_tone)

non_matching_indices, non_matching_times, alignment_lag

In [ ]:
aux_tone - aux_tone[0]

In [ ]:
bpod_trial-bpod_trial[1]

In [ ]:
non_matching_times, alignment_lag = align_timeseries_and_find_non_matching_events(bpod_trial, aux_tone)

In [ ]:
import numpy as np
from scipy.signal import correlate

def align_timeseries(series1, series2):
    """
    This function aligns two timeseries data and finds non-matching events.

    :param series1: First timeseries data as a numpy array.
    :param series2: Second timeseries data as a numpy array.
    :return: Indices of non-matching events in series1.
    """

    # Perform cross-correlation
    correlation = correlate(series1, series2, mode='full')
    # Find the lag that maximizes the cross-correlation
    lag = np.argmax(correlation) - (len(series2) - 1)

    # Align the series by shifting series2
    aligned_series2 = np.roll(series2, -lag)

    # Find non-matching events (where one series has an event and the other does not)
    non_matching_events = np.where((series1 != 0) != (aligned_series2[:len(series1)] != 0))[0]

    return non_matching_events, lag, aligned_series2

# Call the alignment function
non_matching_indices, calculated_lag, aligned_series2 = align_timeseries(bpod_trial, aux_tone)

print(f"Non-matching event indices in series1: {non_matching_indices}")
print(f"Lag for alignment: {calculated_lag}")

In [ ]:
aux_tone

In [ ]:
bpod_trial = bpod_object.session_data["TrialStartTimestamp"]
bpod_trial = np.concatenate(([1], bpod_trial))
ones = np.ones_like(bpod_trial)

ones2 = np.ones_like(aligned_series2)

aux_tone = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
onesfive = np.ones_like(aux_tone) -.5

fig, axs = plt.subplots(2,figsize=(20, 6))
axs[0].scatter(aux_tone, onesfive+0.5)
axs[1].scatter(bpod_trial, ones)
axs[0].scatter(aligned_series2, ones2, c='r')

In [ ]:
from dtw import *
import matplotlib.pyplot as plt

# Create the alignment
alignment = dtw(aux_tone, bpod_trial, keep_internals=True)

# Plot the original time series
plt.figure(figsize=(20, 6))
plt.subplot(3, 1, 1)
plt.plot(aux_tone, label='aux_tone')
plt.plot(bpod_trial, label='bpod_trial')
plt.legend()

# Plot the warping path
plt.subplot(3, 1, 2)
alignment.plot(type="twoway",offset=-2)

# Plot the aligned time series
plt.subplot(3, 1, 3)
plt.plot(aux_tone[alignment.index1], label='aligned aux_tone')
plt.plot(bpod_trial[alignment.index2], label='aligned bpod_trial')
plt.legend()

plt.show()

In [ ]:
alignment.plot()

In [ ]:

def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix


In [ ]:
time_series_1 = aux_tone
time_series_2 = bpod_trial
dtw(time_series_1, time_series_2, 100)

In [ ]:
import numpy as np

def dtw_alignment(ts_a, ts_b, d=lambda x, y: abs(x - y)):
    """Computes the Dynamic Time Warping (DTW) alignment between two time series.

    Args:
        ts_a: First time series (list or array).
        ts_b: Second time series (list or array).
        d: Distance function. Defaults to absolute difference.

    Returns:
        path: The optimal path for the alignment.
        DTW: The full DTW matrix.
    """
    # Create cost matrix with infinite values
    DTW = np.full((len(ts_a) + 1, len(ts_b) + 1), np.inf)
    DTW[0, 0] = 0

    # Populate the cost matrix
    for i in range(1, len(ts_a) + 1):
        for j in range(1, len(ts_b) + 1):
            cost = d(ts_a[i-1], ts_b[j-1])
            DTW[i, j] = cost + min(DTW[i-1, j],    # insertion
                                   DTW[i, j-1],    # deletion
                                   DTW[i-1, j-1])  # match

    # Backtrack to find the optimal path
    path = []
    i, j = len(ts_a), len(ts_b)
    while i > 0 and j > 0:
        path.append((i-1, j-1))
        min_index = np.argmin((DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1]))
        if min_index == 0:
            i -= 1
        elif min_index == 1:
            j -= 1
        else:  # min_index == 2
            i -= 1
            j -= 1

    # Include the start cell in the path
    path.append((0, 0))

    path.reverse()
    return path, DTW

# Example time series data
ts_a = aux_tone
ts_b = bpod_trial

# Get the alignment and DTW matrix
alignment_path, DTW_matrix = dtw_alignment(ts_a, ts_b)

# The alignment path is returned as tuples of aligned indices
alignment_path


In [ ]:
bpod_trial

In [ ]:
aux_tone

## Run image processing jobs


In [ ]:
imaging.Processing.populate(**populate_settings)

### Curation

In [ ]:
imaging.Curation().create1_from_processing_task({'session_id': sessi, 'scan_id': scansi, "paramset_idx": selected_s2pparms_index, "manual_curation": 0})

imaging.MotionCorrection.populate(**populate_settings)

imaging.Segmentation.populate(**populate_settings)

imaging.MaskClassification.populate(**populate_settings)

imaging.Fluorescence.populate(**populate_settings)

imaging.Activity.populate(**populate_settings)

In [ ]:
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
curation_key = (imaging.Curation & scan_key & 'curation_id=1').fetch1('KEY')

In [ ]:
scan_key


## Make Movie

In [ ]:
def make_overview_movies_rspace(curation_key):
    # params_key = (imaging.ProcessingParamSet & 'paramset_idx = "4"').fetch('KEY')
    # reg_tiffs_available = (imaging.ProcessingParamSet & params_key).fetch("params")[0]['reg_tif']
    from scipy.ndimage import mean
    import tifffile
    path = (scan.ScanPath & curation_key).fetch1("path") + ("/suite2p/plane0/reg_tif")

    # path = '/datajoint-data/data/jisooj/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3/suite2p_exp9FGLEFJ3/suite2p/plane0/reg_tif'
    # Get a list of all tiff files in the folder
    tiff_files = [os.path.join(path, f) for f in natsorted(os.listdir(path)) if f.endswith('.tif')]

    # print(tiff_files)

    # Load each tiff stack into a list of numpy arrays
    stacks = []
    for f in tiff_files:
        with tifffile.TiffFile(f) as tif:
            # Get the number of pages in the file
            num_pages = len(tif.pages)
            
            # Create a numpy array to store all pages
            stack = np.zeros((num_pages,) + tif.pages[0].shape, dtype=tif.pages[0].dtype)
            
            # Iterate over the pages and store them in the array
            for i, page in enumerate(tif.pages):
                stack[i] = page.asarray()

        stacks.append(stack)

    # Concatenate the stacks into a single numpy array along the z-axis
    volume = np.concatenate(stacks, axis=0)

    # delete registration tiff
    for f in tiff_files:
        os.remove(f) 
    
    ### moving average filter
    # Create a running Z mean projection of the volume

    runav = 30
    # running_z_projection = uniform_filter_mt(volume, size=(runav,xyrunav,xyrunav))
    running_z_projection = sh.rolling_average_filter(volume, runav)

    session_id = curation_key['session_id']
    scan_id = curation_key['scan_id']

    filename = os.path.join(path, 'registered_movie_' + session_id + '_' + scan_id + '_' + str(runav) + '_frame_runningaverage2' + '.mp4')

    fps = 120   # frames per second - 120 default
    p1 = 2       # percentile scaling low - 1 default
    p2 = 99.998  # percentile scaling high - 99.995 default

    rescaled_image_8bit = sh.make_stack_movie(running_z_projection, filename, fps, p1, p2)

    tmpdir = dj.config['custom'].get('suite2p_fast_tmp')[0]


In [ ]:
make_overview_movies_rspace(curation_key)

## Pose estimation

In [ ]:
dj.Diagram(model) + dj.Diagram(equipment)

In [25]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]
selected_DLCmodel = 'Topcam_2bin_with_scope'
print('- - - -')
print('DLC pose estimation:', scansi)

# insert TOP movie into model table
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])

key = {'session_id': scan_key["session_id"],
    'recording_id': scan_key["scan_id"], 
    'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
    }
model.VideoRecording.insert1(key, skip_duplicates=True)

key.update({'file_path': moviepath,
            'file_id': 0})  #INCREMENT FILE_ID WITH CAM NUMBER?

model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)
model.RecordingInfo.populate()

- - - -
DLC pose estimation: scan9FKWT95S


In [26]:

model.VideoRecording.File * model.VideoRecording() &  'recording_id = "' + scansi + '"'

session_id,recording_id,file_id,"file_path filepath of video, relative to root data directory",camera
sess9FKWT95S,scan9FKWT95S,0,/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4,mini2p1_top


In [27]:
key =  (model.VideoRecording & f'recording_id="{scansi}"').fetch1('KEY')
key.update({'model_name': selected_DLCmodel, 'task_mode': 'trigger'}) 
key      


{'session_id': 'sess9FKWT95S',
 'recording_id': 'scan9FKWT95S',
 'model_name': 'Topcam_2bin_with_scope',
 'task_mode': 'trigger'}

In [28]:
# INSERT pose estimation task
model.PoseEstimationTask.insert_estimation_task(key, key["model_name"], analyze_videos_params={'save_as_csv':True, 'dynamic':(True,.5,60)}) # dynamic cropping

In [29]:
model.PoseEstimationTask() * model.VideoRecording * model.VideoRecording.File()

session_id,recording_id,model_name User-friendly model name,file_id,task_mode load results or trigger computation,pose_estimation_output_dir output dir relative to the root dir,"pose_estimation_params analyze_videos params, if not default",camera,"file_path filepath of video, relative to root data directory"
sess9FKWT95S,scan9FKWT95S,Topcam_2bin_with_scope,0,trigger,/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/device_mini2p1_top_recording_scan9FKWT95S_model_Topcam_2bin_with_scope,=BLOB=,mini2p1_top,/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4


In [30]:
model.RecordingInfo()

session_id,recording_id,px_height height in pixels,px_width width in pixels,nframes number of frames,fps (Hz) frames per second,recording_datetime Datetime for the start of the recording,recording_duration video duration (s) from nframes / fps
sess9FKW82R3,scan9FKW82R3,500,500,36031,60,None,600.517
sess9FKWT95S,scan9FKWT95S,500,500,36056,60,None,600.933
sess9FKWTLNS,scan9FKWTLNS,500,500,36600,60,None,610.0


In [31]:
# run pose estimation
model.PoseEstimation.populate()

2023-11-15 09:29:27.504058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 09:29:27.629366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tobiasr/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-11-15 09:29:27.629388: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-15 09:29:27.659084: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already 

Loading DLC 2.3.4...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Using snapshot-120000 for model /datajoint-data/models/tobiasr/NK_DLC_tracking/Topcam_2bin_withscope-NK-2023-08-31/dlc-models/iteration-0/Topcam_2bin_withscopeAug31-trainset95shuffle1
Starting analysis in dynamic cropping mode with parameters: (True, 0.5, 60)
Switching batchsize to 1, num_outputs (per animal) to 1 and TFGPUinference to False (all these features are not supported in this mode).


/home/tobiasr/miniconda3/envs/datajoint-DLCbackup/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-11-15 09:29:32.495288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tobiasr/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-11-15 09:29:32.495312: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-15 09:29:32.495332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tatchu3): /proc/driver/nvidia/version does not exist
2023-11-15 09:29:32.495584: I tensorflow/core/platform/cpu_featu

Starting to analyze %  /datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4
Loading  /datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4
Duration of video [s]:  600.93 , recorded with  60.0 fps!
Overall # of frames:  36056  found with (before cropping) frame dimensions:  500 500
Starting to extract posture


100%|█████████▉| 36000/36056 [13:11<00:01, 45.49it/s]


Saving results in /datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/device_mini2p1_top_recording_scan9FKWT95S_model_Topcam_2bin_with_scope...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


#### make labeled video

In [36]:
from deeplabcut.utils.make_labeled_video import create_labeled_video
import yaml
from element_interface.utils import find_full_path
from adamacs.paths import get_dlc_root_data_dir


destfolder = model.PoseEstimationTask.infer_output_dir(key)
destfolder

video_path = find_full_path( # Fetch the full video path
    get_dlc_root_data_dir(), ((model.VideoRecording.File & key).fetch1("file_path"))
)

config_paths = sorted( # Of configs in the project path, defer to the datajoint-saved
    list(
        find_full_path(
            get_dlc_root_data_dir(), ((model.Model & key).fetch1("project_path"))
        ).glob("*.y*ml")
    )
)

create_labeled_video( # Pass strings to label the video
    config=str(config_paths[-1]),
    videos=str(video_path),
    destfolder=str(destfolder),
)

# list(list(pathlib.Path((model.VideoRecording.File & key).fetch1("file_path")).parent.glob("device*"))[0].glob("*.y*ml"))

Starting to process video: /datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4
Loading /datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4 and data.
Duration of video [s]: 600.93, recorded with 60.0 fps!
Overall # of frames: 36056 with cropped frame dimensions: 500 500
Generating frames and creating video.


100%|██████████| 36056/36056 [01:27<00:00, 410.01it/s]


[True]

In [40]:
import glob
mp4_files = glob.glob(f"{destfolder}/*.mp4")

In [41]:
mp4_files

['/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/device_mini2p1_top_recording_scan9FKWT95S_model_Topcam_2bin_with_scope/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52DLC_resnet50_Topcam_2bin_withscopeAug31shuffle1_120000_labeled.mp4']

In [32]:
(model.PoseEstimation() * session.Session * session.SessionUser * subject.User()).fetch(format = "frame", order_by = "session_datetime")

,,,,pose_estimation_time,subject,session_datetime,name,shorthand,initials,email,lab
session_id,recording_id,model_name,user_id,,,,,,,,
sess9FKWT95S,scan9FKWT95S,Topcam_2bin_with_scope,6,2023-11-15 09:42:45,ROS-1627,2023-11-03,Jung Jisoo,jisooj,JJ,jjun1@uni-bonn.de,Rose


In [33]:
scansi = "scan9FKWT95S"
scan_key = (model.PoseEstimation & f'recording_id = "{scansi}"').fetch('KEY')[0] 
path = (model.VideoRecording.File & scan_key).fetch("file_path")
path

array(['/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-11-03_scan9FKWT95S_sess9FKWT95S/scan9FKWT95S_mini2p1_top_video_2023-11-03T15_22_52.mp4'],
      dtype=object)

In [34]:
(model.PoseEstimation.BodyPartPosition() & scan_key & 'body_part = "head_middle"').fetch("x_pos", "y_pos")

[array([array([248.18189764, 249.82993031, 249.69035816, ..., 478.01813698,
               477.77332449, 477.69083166])                                ],
       dtype=object),
 array([array([ 32.17335296,  28.48447275,  31.41013908, ..., 227.96029496,
               227.80529046, 225.29934216])                                ],
       dtype=object)]

## Cleaning. Use with caution!

In [ ]:
imaging.ProcessingTask()

In [ ]:
# subject.Subject.delete()
# session.Session.delete()
# imaging.Processing.delete()
# imaging.Curation.delete()
# event.Event.delete()
# event.BehaviorRecording.delete()

In [23]:
key = (session.Session & "session_datetime >= '2023-10-01'").fetch('KEY')
# (session.Session & "session_datetime >= '2023-10-01'").delete()
# (imaging.ProcessingTask & "session_id LIKE 'sess9FKWT2RT%'").delete()
# (imaging.ProcessingTask  & key).delete()
(model.PoseEstimationTask & key).delete()
# (session.Session & "session_id LIKE 'sess9FKWT2RT%'").delete()
# (subject.Subject & "subject = 'ROS-1571'").delete()
# # subject.Subject.delete()
# # session.Session.delete()
# # imaging.Processing.delete()
# # imaging.Curation.delete()
# # event.Event.delete()
# # event.BehaviorRecording.delete()

[2023-11-15 09:28:59,495][INFO]: Deleting 14 rows from `tobiasr02_model`.`__pose_estimation__body_part_position`
[2023-11-15 09:28:59,498][INFO]: Deleting 2 rows from `tobiasr02_model`.`__pose_estimation`
[2023-11-15 09:28:59,500][INFO]: Deleting 3 rows from `tobiasr02_model`.`pose_estimation_task`
[2023-11-15 09:29:05,680][INFO]: Deletes committed.


3

In [ ]:
session.Session.delete()

In [ ]:
# session.Session.drop()
# scan.Scan.drop()
# imaging.Processing.drop()
# imaging.Curation.drop()